<a href="https://colab.research.google.com/github/lcremoni/Data_Science/blob/master/Analisi_Coldiretti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from google.colab import drive
from bokeh.plotting import figure 
from bokeh.io import output_notebook, show
from datetime import timedelta
from bokeh.models import DatetimeTickFormatter
from bokeh.palettes import Category20

output_notebook()

#drive.mount('/content/gdrive')

In [0]:
path = '/content/drive/My Drive/Colab Notebooks/'
#path = '/content/'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Analisi Modelli Unico

In [4]:
#@title Numero pratiche per stato
d = pd.read_excel(path+'16-09-19 Unico PF.xlsx')
d = d[['Zona','descr. Zona','Operatore ','data apertura pratica','Data Impegno','Stato Dichiarazione']]
d.columns = ['zona','descr_zona','operatore','data_apertura','data','stato']
d.groupby('stato').count()


,zona,descr_zona,operatore,data_apertura,data
stato,,,,,
Dichiarazione aperta,267,267,267,267,19
Dichiarazione chiusa,133,133,133,133,133
Dichiarazione stampata,4820,4820,4820,4820,4820


In [5]:
#@title Numero pratiche chiuse per zona
d['unico'] = d.stato.apply(lambda x: 1 if x in ['Dichiarazione chiusa','Dichiarazione stampata'] else 0)
d = d.loc[d['unico'] == 1]
d.groupby('descr_zona').count()

,zona,operatore,data_apertura,data,stato,unico
descr_zona,,,,,,
BRENO,284,284,284,284,284,284
BRESCIA DUE,428,428,428,428,428,428
BRESCIA UNO,40,40,40,40,40,40
CHIARI,425,425,425,425,425,425
ISEO,261,261,261,261,261,261
LENO,500,500,500,500,500,500
LONATO,654,654,654,654,654,654
MONTICHIARI,551,551,551,551,551,551
ORZINUOVI,626,626,626,626,626,626


In [6]:
#@title dati statistici numero pratiche per zona
d.groupby('descr_zona')['zona'].count().describe()

count     14.000000
mean     353.785714
std      199.390679
min       11.000000
25%      266.750000
50%      359.000000
75%      482.000000
max      654.000000
Name: zona, dtype: float64

In [7]:
#@title Grafico pratiche Coldiretti

g_data = d.groupby('data').count()

giorni = g_data.index
p = figure(title="Sistema Coldiretti", x_axis_type='datetime', plot_width=1000)
p.vbar(x=giorni, bottom=0, top=g_data['unico'], width=timedelta(days=0.8), 
       fill_alpha=1, color=Category20[4][0], line_color=None, legend="Numero Modelli Unico")
p.legend.location = "top_left"
p.xaxis[0].formatter.days = '%d/%m'
p.xaxis.major_label_orientation = 3.14/3
#p.xaxis.formatter=DatetimeTickFormatter(days = ['%d/%m'])
show(p)

In [8]:
g_data['unico'].describe()

count     91.000000
mean      54.428571
std       60.996019
min        1.000000
25%        9.000000
50%       31.000000
75%       85.500000
max      238.000000
Name: unico, dtype: float64

# Analisi Modelli unico per zona

In [0]:

Zona = "LENO"  #@param ['LONATO', 'ROVATO', 'BRESCIA-IMPRESA VERDE BRESCIA SRL','VEROLANUOVA', 'BRENO', 'LENO', 'CHIARI', 'SALO', 'VESTONE','ORZINUOVI', 'MONTICHIARI', 'SEDE PROVINCIALE', 'ISEO', 'BRESCIA']


In [10]:
#@title Grafico pratiche zona
d['descr_zona'] = d['descr_zona'].str.strip().replace({'SALO\'':'SALO','BRESCIA UNO':'BRESCIA','BRESCIA DUE':'BRESCIA-IMPRESA VERDE BRESCIA SRL'})
d_pivot = pd.pivot_table(d, values='unico', index=['descr_zona', 'operatore','data'], aggfunc=sum)
d_zona = d_pivot.loc[Zona].groupby('data').sum()
giorni = d_zona.index
p = figure(title=Zona, x_axis_type='datetime', plot_width=1000)
p.vbar(x=giorni, bottom=0, top=d_zona['unico'], width=timedelta(days=0.8), 
       fill_alpha=1, color=Category20[4][0], line_color=None, legend="Numero Modelli Unico")
p.legend.location = "top_left"
p.xaxis[0].formatter.days = '%d/%m'
p.xaxis.major_label_orientation = 3.14/3
#p.xaxis.formatter=DatetimeTickFormatter(days = ['%d/%m'])
show(p)

In [11]:
#@title Dati statistici zona
d_zona.describe()

,unico
count,30.000000
mean,16.666667
std,15.182416
min,1.000000
25%,5.000000
50%,13.000000
75%,23.750000
max,57.000000


# Analisi Modelli 730

In [12]:
#@title Numero 730 per zona
d7 = pd.read_csv(path+'Rawdata730.csv', low_memory=False)
d7 = d7[:-1]
d7.at[1801, 'desc_ufficio'] = 'BRESCIA-IMPRESA VERDE BRESCIA SRL'
d7 = d7[['desc_ufficio','Ord_puliti','utente','data elaborazione']]
d7 = d7.loc[d7['Ord_puliti'] == 1]
d7.columns = ['zona','730','utente','data']
d7['zona'] = d7['zona'].str.strip().replace({'SALO\'':'SALO'})
d7['data'] = pd.to_datetime(d7['data'], dayfirst=True)
d7.groupby('zona').count()

,730,utente,data
zona,,,
BRENO,357,357,357
BRESCIA,1062,1062,1062
BRESCIA-IMPRESA VERDE BRESCIA SRL,1446,1446,1446
CHIARI,690,690,690
ISEO,433,433,433
LENO,809,809,809
LONATO,1221,1221,1221
MONTICHIARI,1337,1337,1337
ORZINUOVI,708,708,708


In [13]:
#@title Dati statistici 730 per zona
d7.groupby('zona').count().describe()

,730,utente,data
count,13.000000,13.000000,13.000000
mean,841.230769,841.230769,841.230769
std,347.377353,347.377353,347.377353
min,357.000000,357.000000,357.000000
25%,609.000000,609.000000,609.000000
50%,809.000000,809.000000,809.000000
75%,1062.000000,1062.000000,1062.000000
max,1446.000000,1446.000000,1446.000000


In [14]:
#@title Numero 730 per zona e utente
d7.groupby(['zona','utente']).count()

730  data
zona                              utente                              
BRENO                             AICMI.BSTERESINAR.00865    319   319
                                  AICMI.BSVINCENZAM.00865     38    38
BRESCIA                           AICMI.BSANDREAP.00865        2     2
                                  AICMI.BSCRISTINAF.00865    208   208
                                  AICMI.BSELIAR.00865        371   371
                                  AICMI.BSGIANMARIAR.00865     6     6
                                  AICMI.BSGIULIAP.00865      284   284
                                  AICMI.BSMARIAM.00865         9     9
                                  AICMI.BSSTEFANOC.00865     180   180
                                  BLUARANCIO_TEST              2     2
BRESCIA-IMPRESA VERDE BRESCIA SRL AICMI.BSANDREAP.00865       65    65
                                  AICMI.BSANTONIOZ.00865      36    36
                                  AICMI.BSGIULIANOR.00865    453   453
                                  AICMI.BSMARIAM.00865       765   765
                                  AICMI.BSPIERFAUSTOS.00865    1     1
                                  AICMI.BSROBERTOP.00865     124   124
                                  BLUARANCIO_TEST              2     2
CHIARI                            AICMI.BSANDREAP.00865        1     1
                                  AICMI.BSPIERFAUSTOS.00865  438   438
                                  AICMI.BSSILVIAM.00865      251   251
ISEO                              AICMI.BSALBERTOF.00865      51    51
                                  AICMI.BSDAVIDEA.00865      242   242
                                  AICMI.BSELIAR.00865         24    24
                                  AICMI.BSEMANUELEZ.00865     25    25
                                  AICMI.BSSTEFANOC.00865      19    19
                                  AICMI.BSVERONICAF.00865     71    71
                                  BLUARANCIO_TEST              1     1
LENO                              AICMI.BSANDREAP.00865        1     1
                                  AICMI.BSERIKASC.00865      217   217
                                  AICMI.BSGIUSEPPINAM.00865  263   263
...                                                          ...   ...
                                  AICMI.BSSTEFANIAB.00865    173   173
LONATO                            AICMI.BSELISAD.00865       412   412
                                  AICMI.BSFRANCESCAL.00865   184   184
                                  AICMI.BSLAURAB.00865       191   191
                                  AICMI.BSMICHELAM.00865      21    21
                                  AICMI.BSPAOLAG.00865       413   413
MONTICHIARI                       AICMI.BSGIANLUCAF.00865    379   379
                                  AICMI.BSLAURAAL.00865       77    77
                                  AICMI.BSMICHELADA.00865    881   881
ORZINUOVI                         AICMI.BSANDREAP.00865        1     1
                                  AICMI.BSELISAZ.00865        29    29
                                  AICMI.BSMARCOF.00865       428   428
                                  AICMI.BSMASSIMOF.00865      45    45
                                  AICMI.BSPRIMOA.00865       205   205
ROVATO                            AICMI.BSCHIARAPE.00865     328   328
                                  AICMI.BSDONATELLAF.00865    66    66
                                  AICMI.BSVERONICAGH.00865   420   420
SALO                              AICMI.BSALICED.00865       487   487
                                  AICMI.BSCHIARAZ.00865      104   104
                                  AICMI.BSMARIAM.00865        18    18
VEROLANUOVA                       AICMI.BSANNALISAC.00865    412   412
                                  AICMI.BSELIAR.00865         43    43
                                  AICMI.BSFLAVIAL.00865      159   159
                                  AICMI.BSJESSICATO.00865    329   329
              

In [15]:
#@title Dati statistici numero 730 per zona e utente
d7.groupby(['zona','utente']).count().describe()

,730,data
count,61.000000,61.000000
mean,179.278689,179.278689
std,192.793165,192.793165
min,1.000000,1.000000
25%,25.000000,25.000000
50%,124.000000,124.000000
75%,284.000000,284.000000
max,881.000000,881.000000


In [16]:
#@title Dati statistici numero 730 per zona/utente/data
d7.groupby(['zona','utente','data']).count().describe()

,730
count,1539.000000
mean,7.105913
std,6.463448
min,1.000000
25%,2.000000
50%,5.000000
75%,10.000000
max,79.000000


In [17]:
#@title Grafico 730 sistema
g7_data = d7.groupby('data').count()

g7_data = g7_data[['730']]
giorni = g7_data.index
p = figure(title='Analisi 730 Sistema', x_axis_type='datetime', plot_width=1000)
p.vbar(x=giorni, bottom=0, top=g7_data['730'], width=timedelta(days=0.8), 
       fill_alpha=1, color=Category20[4][2], line_color=None, legend="Numero 730")
p.legend.location = "top_left"
p.xaxis[0].formatter.days = '%d/%m'
p.xaxis.major_label_orientation = 3.14/3
#p.xaxis.formatter=DatetimeTickFormatter(days = ['%d/%m'])
show(p)

In [18]:
#@title Dati statistici 730 sistema
g7_data.describe()

,730
count,90.000000
mean,121.511111
std,91.568265
min,1.000000
25%,27.500000
50%,119.000000
75%,197.000000
max,296.000000


In [0]:
g_data = g_data['unico']

In [0]:
gtot_data = pd.merge(g_data, g7_data, left_index=True, right_index=True, how='outer')

In [0]:
gtot_data['date'] = gtot_data.index

In [0]:
#gtot_data = gtot_data.fillna(0)

In [23]:
from bokeh.core.properties import value
from bokeh.models.tools import HoverTool
modelli = ["730", "unico"]
colors = [Category20[4][2],Category20[4][0] ]

p = figure(title='Analisi Campagne Sistema', x_axis_type='datetime', plot_width=1000)
#giorni = gtot_data.data
ht = HoverTool(tooltips=[("data", "@data{%d/%m/%Y}"), ("unico", "@unico")],
          formatters={'data': 'datetime'})
p.add_tools(ht)
p.vbar_stack(modelli, x='date', width=timedelta(days=0.8), color=colors, source=gtot_data, 
             legend=[value(x) for x in modelli])

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)

# Analisi Modelli Unico SP

In [0]:
ds = pd.read_excel(path+'16-09-19 Unico SP.xlsx')

In [0]:
ds = ds[['Zona','descr. Zona','Operatore','data apertura pratica','Data Impegno','Stato Dichiarazione']]
ds.columns = ['zona','descr_zona','operatore','data_apertura','data','stato']
ds['unico'] = ds.stato.apply(lambda x: 1 if x in ['Dichiarazione chiusa','Dichiarazione stampata'] else 0)
ds['descr_zona'] = ds['descr_zona'].str.strip().replace({'SALO\'':'SALO','BRESCIA UNO':'BRESCIA','BRESCIA DUE':'BRESCIA-IMPRESA VERDE BRESCIA SRL'})
ds = ds.loc[ds['unico'] == 1]
gs_data = ds.groupby('data').count()

In [26]:
ds['descr_zona'].unique()

array(['LONATO', 'BRESCIA', 'LENO', 'MONTICHIARI', 'ORZINUOVI',
       'VEROLANUOVA', 'BRESCIA-IMPRESA VERDE BRESCIA SRL', 'CHIARI',
       'SALO', 'ISEO', 'ROVATO', 'BRENO'], dtype=object)

In [27]:
giorni = gs_data.index
p = figure(title="Sistema Coldiretti", x_axis_type='datetime', plot_width=1000)
p.vbar(x=giorni, bottom=0, top=gs_data['unico'], width=timedelta(days=0.8), 
       fill_alpha=1, color=Category20[4][1], line_color=None, legend="Numero Modelli Unico SP")
p.legend.location = "top_left"
p.xaxis[0].formatter.days = '%d/%m'
p.xaxis.major_label_orientation = 3.14/3
#p.xaxis.formatter=DatetimeTickFormatter(days = ['%d/%m'])
show(p)

# Analisi cumulata sistema

In [0]:
gs_data = gs_data['unico']
gs_data.columns = ['unico_sp']
gtots_data = pd.merge(gtot_data, gs_data, left_index=True, right_index=True, how='outer')
#gtots_data = gtots_data.fillna(0)
gtots_data.columns = ['unico PF','730','date','unico SP']

In [29]:
gtots_data.describe()

,unico PF,730,unico SP
count,91.000000,90.000000,69.000000
mean,54.428571,121.511111,9.985507
std,60.996019,91.568265,9.807699
min,1.000000,1.000000,1.000000
25%,9.000000,27.500000,2.000000
50%,31.000000,119.000000,7.000000
75%,85.500000,197.000000,14.000000
max,238.000000,296.000000,39.000000


In [0]:
gtots_data = gtots_data.fillna(0)

In [31]:
#from bokeh.resources import CDN
#from bokeh.embed import file_html

modelli = ["730", "unico PF", "unico SP"]
colors = [Category20[4][2],Category20[4][0],Category20[4][1] ]

p = figure(title='Analisi Campagne Sistema', x_axis_type='datetime', plot_width=1000)

#giorni = gtot_data.data
p.vbar_stack(modelli, x='date', width=timedelta(days=0.8), color=colors, source=gtots_data, 
             legend=[value(x) for x in modelli])
ht = HoverTool(tooltips=[("data", "@data{%d/%m/%Y}"), ("730", "@730"), ("unico PF", "@{unico PF}"), ("unico SP", "@{unico SP}")],
          formatters={'data': 'datetime'})
p.add_tools(ht)
p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

#html = file_html(p, CDN, "my plot")
show(p)

# Analisi cumulata per zona

In [0]:

Zona = "BRESCIA-IMPRESA VERDE BRESCIA SRL"  #@param ['LONATO', 'BRESCIA', 'LENO', 'MONTICHIARI', 'ORZINUOVI','VEROLANUOVA', 'BRESCIA-IMPRESA VERDE BRESCIA SRL', 'CHIARI','SALO', 'ISEO', 'ROVATO', 'BRENO']


In [0]:
d7_pivot = pd.pivot_table(d7, values='730', index=['zona', 'utente','data'], aggfunc=sum)
try: 
    d7_zona = d7_pivot.loc[Zona].groupby('data').sum()
    pass

except KeyError:
    
    pass

ds_pivot = pd.pivot_table(ds, values='unico', index=['descr_zona', 'operatore','data'], aggfunc=sum)


In [0]:
#indice_zone = ['LONATO', 'ROVATO', 'BRESCIA-IMPRESA VERDE BRESCIA SRL','VEROLANUOVA', 'BRENO', 'LENO', 'CHIARI', 'SALO', 'VESTONE','ORZINUOVI', 'MONTICHIARI', 'SEDE PROVINCIALE', 'ISEO', 'BRESCIA']

In [0]:
#ds_pivot = ds_pivot.reindex(indice_zone, level=0)

In [0]:
try: 
    d_zona = d_pivot.loc[Zona].groupby('data').sum()
    dtot_zona = pd.merge(d7_zona, d_zona, left_index=True, right_index=True, how='outer')
    pass

except KeyError:
    dtot_zona['unico PF']=0
    pass
try:
    ds_zona = ds_pivot.loc[Zona].groupby('data').sum()
    dtot_zona = pd.merge(dtot_zona, ds_zona, left_index=True, right_index=True, how='outer')
    pass

except KeyError:
    dtot_zona['unico SP']=0
    pass

In [0]:
dtot_zona.columns = ['730','unico PF','unico SP']

In [38]:
dtot_zona.describe()

,730,unico PF,unico SP
count,61.000000,53.000000,31.000000
mean,23.704918,8.075472,2.225806
std,19.979443,5.777440,1.359158
min,1.000000,1.000000,1.000000
25%,10.000000,3.000000,1.000000
50%,17.000000,7.000000,2.000000
75%,34.000000,12.000000,3.000000
max,79.000000,23.000000,5.000000


In [0]:
dtot_zona = dtot_zona.fillna(0)

In [40]:
modelli = ["730", "unico PF", "unico SP"]
colors = [Category20[4][2],Category20[4][0],Category20[4][1] ]

p = figure(title='Analisi Zona '+Zona, x_axis_type='datetime', plot_width=1000)

#giorni = gtot_data.data
p.vbar_stack(modelli, x='data', width=timedelta(days=0.8), color=colors, source=dtot_zona, 
             legend=[value(x) for x in modelli])
ht = HoverTool(tooltips=[("data", "@data{%d/%m/%Y}"), ("730", "@730"), ("unico PF", "@{unico PF}"), ("unico SP", "@{unico SP}")],
          formatters={'data': 'datetime'})
p.add_tools(ht)
p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p) 

In [0]:
#from google.colab import files
#d_pivot.loc[Zona].to_csv('output.csv')
#files.download('output.csv')

# Analisi attività 730 per zone

In [0]:
d7_zonadata = d7.groupby(['zona','data']).count()

In [0]:
d7_zonadata['zona'] = d7_zonadata.index.get_level_values(level=0)
d7_zonadata['data'] = d7_zonadata.index.get_level_values(level=1)

In [0]:
d7_zonadata.data = d7_zonadata.data.astype(str)

In [0]:
index = pd.date_range(d7_zonadata.data.min(), d7_zonadata.data.max())

In [0]:
multi_index = pd.MultiIndex.from_product([d7_zonadata.zona.unique(), index])

In [0]:
d7_zonadata = d7_zonadata.reindex(multi_index)


In [0]:
d7_zonadata['data'] = d7_zonadata.index.get_level_values(level=1)
d7_zonadata['zona'] = d7_zonadata.index.get_level_values(level=0)
d7_zonadata = d7_zonadata.fillna(0)

In [0]:
d7_zonadata.data = d7_zonadata.data.astype(str)

In [64]:
list(reversed(Blues[9]))

['#f7fbff',
 '#deebf7',
 '#c6dbef',
 '#9ecae1',
 '#6baed6',
 '#4292c6',
 '#2171b5',
 '#08519c',
 '#08306b']

In [69]:
from bokeh.models import BasicTicker, ColorBar, ColumnDataSource, LinearColorMapper, PrintfTickFormatter
from bokeh.transform import transform
from bokeh.palettes import GnBu

#data.Year = data.Year.astype(str)
#data = data.set_index('Year')
#data.drop('Annual', axis=1, inplace=True)
#data.columns.name = 'Month'

# reshape to 1D array or rates with a month and year for each row.
#df = pd.DataFrame(data.stack(), columns=['rate']).reset_index()

source = d7_zonadata

# this is the colormap from the original NYTimes plot
#colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
colors = list(reversed(GnBu[9]))
mapper = LinearColorMapper(palette=colors, low=d7_zonadata['730'].min(), high=d7_zonadata['730'].max())

p = figure(plot_width=1100, plot_height=450, title="Analisi 730 per zona e data",
           x_range=list(index.astype(str)), y_range=list(d7_zonadata.zona.unique()),
           toolbar_location=None, tools="", x_axis_location="above")

p.rect(x="data", y="zona", width=1, height=1, source=source,
       line_color=None, fill_color=transform('730', mapper))

color_bar = ColorBar(color_mapper=mapper, location=(0, 0),
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     formatter=PrintfTickFormatter(format="%d%%"))

p.add_layout(color_bar, 'right')
ht = HoverTool(tooltips=[("zona", "@zona"),("data", "@data{%d/%m/%Y}"), ("730", "@730")],
          formatters={'data': 'datetime'})
p.add_tools(ht)
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "5pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = 1.0

show(p)

# Analisi attività 730 per zona e utente

In [0]:
d7_pivotf = d7_pivot
d7_pivotf['zona'] = d7_pivot.index.get_level_values(level=0)
d7_pivotf['data'] = d7_pivot.index.get_level_values(level=2)
d7_pivotf['utente'] = d7_pivot.index.get_level_values(level=1)


In [54]:
from bokeh.transform import jitter

zone = d7_pivotf['zona'].unique()
source = d7_pivotf

p = figure(plot_width=1100, plot_height=700, y_range=zone, x_axis_type='datetime',
           title="Analisi attività 730 per zona e utente")

#p.circle(x='time', y=jitter('day', width=0.6, range=p.y_range),  source=source, alpha=0.3)
#p.circle(x=jitter('data', 0.6) , y=jitter('zona', width=0.6, range=p.y_range),  source=source, alpha=0.3)
p.circle(x=jitter('data', 0.6) , y=jitter('zona', width=0.6, range=p.y_range),  source=source, alpha=0.3, size='730')

ht = HoverTool(tooltips=[("zona", "@zona"),("utente", "@utente"),("data", "@data{%d/%m/%Y}"), ("730", "@730")],
          formatters={'data': 'datetime'})
p.add_tools(ht)

p.xaxis[0].formatter.days = ['%d/%m']
p.x_range.range_padding = 0
p.ygrid.grid_line_color = None
p.xaxis.major_label_orientation = 1.0

show(p)